# 📊 پیش‌بینی تایید وام با استفاده از یادگیری ماشین
این نوت‌بوک پروژه‌ای کامل برای بررسی واجد شرایط بودن افراد برای دریافت وام است، بر اساس داده‌های loan شامل پردازش، مدل‌سازی، ارزیابی و بهبود.

In [ ]:

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from imblearn.over_sampling import SMOTE
from sklearn.impute import SimpleImputer

# بارگذاری داده
df = pd.read_csv('loan.csv')
df.head()


## 🔍 بررسی و مدیریت مقادیر گمشده

In [ ]:

# بررسی مقادیر گمشده
df.isnull().sum()


In [ ]:

# مدیریت مقادیر گمشده
cat_cols = df.select_dtypes(include='object').columns
num_cols = df.select_dtypes(include=['int64', 'float64']).columns

imputer_cat = SimpleImputer(strategy='most_frequent')
imputer_num = SimpleImputer(strategy='mean')

df[cat_cols] = imputer_cat.fit_transform(df[cat_cols])
df[num_cols] = imputer_num.fit_transform(df[num_cols])


## 🔢 کدگذاری متغیرهای غیر عددی

In [ ]:

le = LabelEncoder()
for col in cat_cols:
    df[col] = le.fit_transform(df[col])


## ✂️ تقسیم داده‌ها به آموزش، اعتبارسنجی و تست

In [ ]:

X = df.drop('Loan_Status', axis=1)
y = df['Loan_Status']

# تقسیم به تست و بقیه
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.15, random_state=42, stratify=y)
# تقسیم آموزش و اعتبارسنجی
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.2353, random_state=42, stratify=y_train_val)


## ⚖️ نرمال‌سازی داده‌های عددی

In [ ]:

scaler = StandardScaler()
X_train_scaled = X_train.copy()
X_val_scaled = X_val.copy()
X_test_scaled = X_test.copy()

X_train_scaled[num_cols] = scaler.fit_transform(X_train[num_cols])
X_val_scaled[num_cols] = scaler.transform(X_val[num_cols])
X_test_scaled[num_cols] = scaler.transform(X_test[num_cols])


## 🛠️ بخش بهبود: مهندسی ویژگی (Total_Income)

In [ ]:

df['Total_Income'] = df['ApplicantIncome'] + df['CoapplicantIncome']


## ⚙️ بخش بهبود: استفاده از SMOTE برای بالانس کلاس‌ها

In [ ]:

sm = SMOTE(random_state=42)
X_resampled, y_resampled = sm.fit_resample(X_train_scaled, y_train)


## 🤖 آموزش مدل‌ها

In [ ]:

models = {
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "KNN": KNeighborsClassifier(n_neighbors=5),
    "ANN": MLPClassifier(hidden_layer_sizes=(10,10), max_iter=1000)
}

for name, model in models.items():
    print(f"\nModel: {name}")
    model.fit(X_resampled, y_resampled)
    preds = model.predict(X_val_scaled)
    print(classification_report(y_val, preds))


## 📊 ارزیابی نهایی مدل‌ها روی داده تست

In [ ]:

for name, model in models.items():
    print(f"\nModel: {name}")
    test_preds = model.predict(X_test_scaled)
    print(classification_report(y_test, test_preds))
